In [ ]:
from bs4 import BeautifulSoup
from requests import get
from random import choice 
from selenium import webdriver  
from selenium.webdriver.chrome.options import Options
import pandas as pd
from tqdm import tqdm
from datetime import datetime, timedelta
import html
import re


In [ ]:
BASE_URL = "https://timesofindia.indiatimes.com"

USER_AGENTS = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.0 Safari/605.1.15",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/117.0",
    "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:109.0) Gecko/20100101 Firefox/117.0",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_3_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:115.0) Gecko/20100101 Firefox/115.0",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 12_6_3) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.3 Safari/605.1.15"
]

news_orgs = [
    "Times of India",
]

fname = "proxyscrape_premium_http_proxies.txt"

PROXIES = []
with open(fname, 'r') as f:
    for line in f:
        PROXIES.append(line.strip())

# print(PROXIES)


INITIAL_START_TIME = 44562  #1/1/2022
START_YEAR = 2022

In [ ]:
#TOI arcive URL generator
def generate_url(target_date, start_year = START_YEAR):
    # Base parameters
    start_date = datetime(start_year, 1, 1)
    
    # Calculate days difference
    days_diff = (target_date - start_date).days
    
    # Calculate starttime for target date
    starttime = INITIAL_START_TIME + days_diff
    
    # Generate URL
    url = f"{BASE_URL}/{target_date.year}/{target_date.month}/{target_date.day}/archivelist/year-{target_date.year},month-{target_date.month},starttime-{starttime}.cms"
    
    return url    

In [ ]:
#Date generator
def generate_dates(start_year = START_YEAR):
    start_date = datetime(start_year, 1, 1)
    end_date = datetime.now()  # Current date
    
    date = start_date
    dates = []
    
    while date <= end_date:
        dates.append(date)
        date += timedelta(days=1)
    
    return dates


In [ ]:
def get_details(url):
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--window-size=1920,1080")
    chrome_options.add_argument("--no-sandbox")
    
    chrome_options.add_argument(f'user-agent={choice(USER_AGENTS)}')
    chrome_options.add_argument(f'--proxy-server=http://{choice(PROXIES)}')

    session = webdriver.Chrome(options=chrome_options)
    try:
        session.get(url)
        # time.sleep(3)  # Time for JS to load

        html = session.page_source
        soup = BeautifulSoup(html, "html.parser")
        # with open("response_log_1.txt", "a", encoding="utf-8") as f:
        #     f.write(soup.prettify() + "\n")

        article_links = []

        for a in soup.find_all("a", href=True):
            if "/articleshow/" in a["href"]:
                article_links.append(a["href"])

        return article_links

    except Exception as e:
        with open ("error_log.txt", "a") as f:
            f.write(f"Error fetching {url}: {e}\n")
        return []
    finally:
        session.quit()

In [15]:
def get_details_basic(url):

    try:
        headers = {
            "User-Agent": choice(USER_AGENTS)
        }
        proxy_addr = choice(PROXIES)
        proxies = {
            "http": f"http://{proxy_addr}",
            "https": f"http://{proxy_addr}"
        }

        response = get(url, headers=headers, proxies=proxies, timeout=5)
        soup = BeautifulSoup(response.text, 'html.parser')
        # with open("response_log.txt", "a", encoding="utf-8") as f:
        #     f.write(soup.prettify() + "\n")

        article_links = []

        for a in soup.find_all("a", href=True):
            if "/articleshow/" in a["href"]:
                article_links.append(a["href"])

        return article_links

    except Exception as e:
        with open ("error_log.txt", "a") as f:
            f.write(f"Error fetching {url}: {e}\n")
        return []


In [ ]:
dates = generate_dates(start_year=START_YEAR)
dates = dates[:5]  #testing with 5 days

urls = []
for date in dates : 
    url = generate_url(date, start_year=START_YEAR)
    urls.append(url)


In [18]:
# get_details_basic('https://timesofindia.indiatimes.com/2022/1/5/archivelist/year-2022,month-1,starttime-44566.cms')
get_details('https://timesofindia.indiatimes.com/2022/1/5/archivelist/year-2022,month-1,starttime-44566.cms')

['/business/india-business/india-to-benefit-from-easier-uk-visa-rules/articleshow/120980145.cms',
 '/business/india-business/under-cyberattack-nse-bse-curb-website-access-from-abroad/articleshow/120980127.cms',
 '/world/rest-of-world/south-korea-says-north-korea-has-fired-several-missiles-towards-its-eastern-waters/articleshow/120980116.cms',
 '/business/international-business/musks-starlink-gets-loi-for-satcom-licence/articleshow/120979946.cms',
 '/india/operation-name-and-two-women-send-powerful-message/articleshow/120979869.cms',
 '/india/hal-on-high-alert-for-iaf-fighter-fleet-care/articleshow/120979494.cms',
 '/india/husbands-killed-by-terrorists-justice-delivered-by-operation-sindoor/articleshow/120979410.cms',
 '/india/muslim-clerics-netas-back-operation-against-pakistan-terror-infra/articleshow/120979319.cms',
 '/rbi-board-needs-to-play-like-rahul-dravid-not-sidhu-raghuram-rajan/articleshow/66524299.cms',
 '/karnataka-bypolls-congress-shocks-bjp-in-ballari-lok-sabha-seat/articl

In [ ]:
all_links = []
for url in tqdm(urls, desc="Fetching article links"):
    print(f"Fetching links from: {url}")

    # #using basic requests.get method to reduce processing time as JS render not needed
    # links = get_details_basic(url)

    links = get_details(url)
    date = dates[urls.index(url)].strftime("%Y-%m-%d")
    for link in links : 
        all_links.append({"link": link, "date" : date})

df = pd.DataFrame(all_links)
df

Fetching article links:   0%|          | 0/5 [00:00<?, ?it/s]

Fetching links from: https://timesofindia.indiatimes.com/2022/1/1/archivelist/year-2022,month-1,starttime-44562.cms


Fetching article links:  20%|██        | 1/5 [00:02<00:11,  2.83s/it]

Fetching links from: https://timesofindia.indiatimes.com/2022/1/2/archivelist/year-2022,month-1,starttime-44563.cms


Fetching article links:  40%|████      | 2/5 [00:05<00:08,  2.88s/it]

...

Fetching links from: https://timesofindia.indiatimes.com/2022/1/4/archivelist/year-2022,month-1,starttime-44565.cms


Fetching article links:  80%|████████  | 4/5 [00:13<00:03,  3.61s/it]

Fetching links from: https://timesofindia.indiatimes.com/2022/1/5/archivelist/year-2022,month-1,starttime-44566.cms


Fetching article links: 100%|██████████| 5/5 [00:15<00:00,  3.18s/it]


,link,date
0,/first-look/detective-byomkesh-bakshy/articles...,2022-01-01
1,/first-look/first-look-of-ranbir-kapoors-roy-a...,2022-01-01
2,/entertainment/english/hollywood/previews/edge...,2022-01-01
3,https://timesofindia.indiatimes.com/sports/cri...,2022-01-01
4,https://timesofindia.indiatimes.com/sports/cri...,2022-01-01
...,...,...
199,https://timesofindia.indiatimes.com/india/oper...,2022-01-05
200,https://timesofindia.indiatimes.com/city/bhuba...,2022-01-05
201,https://timesofindia.indiatimes.com/city/bhopa...,2022-01-05
202,https://timesofindia.indiatimes.com/city/indor...,2022-01-05


In [ ]:
df_1 = df.copy()

df.drop_duplicates(subset=["link"], inplace=True)
df["link"] = df["link"].apply(lambda index: index if "http" in index else BASE_URL + index)
df["publisher"] = "Times of India"


df.to_csv("df.csv", index=False)
df

,link,date,publisher
0,https://timesofindia.indiatimes.com/first-look...,2022-01-01,Times of India
1,https://timesofindia.indiatimes.com/first-look...,2022-01-01,Times of India
2,https://timesofindia.indiatimes.com/entertainm...,2022-01-01,Times of India
3,https://timesofindia.indiatimes.com/sports/cri...,2022-01-01,Times of India
4,https://timesofindia.indiatimes.com/sports/cri...,2022-01-01,Times of India
...,...,...,...
73,https://timesofindia.indiatimes.com/sports/nba...,2022-01-03,Times of India
74,https://timesofindia.indiatimes.com/sports/nba...,2022-01-03,Times of India
75,https://timesofindia.indiatimes.com/sports/nfl...,2022-01-03,Times of India
76,https://timesofindia.indiatimes.com/sports/nfl...,2022-01-03,Times of India


In [44]:
df = pd.read_csv("df.csv")

In [ ]:
def process(title, content):
    def clean_text(text):
        if not (type(text) is str):
            return text

        # 1. Whitespace trimming
        text = text.strip()

        # 2. Converts HTML entities and special characters to plain text
        text = html.unescape(text)

        # 3. Removes <iframe>, <script>, <div>, <style> tags and their content
        text = re.sub(r'<(iframe|script|div|style)[^>]*>.*?</\1>', '', text, flags=re.DOTALL|re.IGNORECASE)

        # 4. Removes promotional/unwanted phrases
        promotions = [
            'click here', 'Click Here', 'advertisement', 'Ad', 'Subscribe', 
            'Read More', 'Learn More', 'Join Now', 'Get Started', 'Sign Up', 
            'Click to Learn More', 'Buy Now', 'Limited Time Offer', 'Act Now', 
            'Don’t Miss Out', 'Exclusive Deal', 'Shop Now', 'Download Now', 
            'Try for Free', 'Free Trial', 'Register Now', 'See More', 
            'Follow Us', 'Stay Updated', 'Get Updates', 'Explore More', 
            'More Info', 'This Just In', 'Breaking News', 'Today’s Deals',
            'YOU MAY LIKE', 'Post comment', 'You can now subscribe to our Economic Times WhatsApp channel',
            'like and share', 'subscribe to our', 'follow us on', 'for more updates',
            'copyright', 'publisher:', 'advertisements', 'SUBSCRIBE NOW!'
        ]
        for phrase in promotions :
            text = text.replace(phrase, '')

        # 5. Removes "views are personal"
        text = re.sub(r'views?\s+are\s+personal', '', text, flags=re.IGNORECASE)

        # 6. Removes JS (function(){...})(...); blocks
        text = re.sub(r'\(function\s*\([^\)]*\)\s*\{.*?\}\s*\)\s*\([^\)]*\)\s*;?', '', text, flags=re.DOTALL)

        # 7. Removes https and www. links
        text = re.sub(r'https?://\S+|www\.\S+', '', text)

        # 8. Removes twitter links (pic.twitter.com)
        text = re.sub(r'pic\.twitter\.com/\S+', '', text)

        # 9. Removes news organization names
        for org in news_orgs:
            text = re.sub(rf'\b{re.escape(org)}\b', '', text, flags=re.IGNORECASE)

        # 10. Remove any remaining HTML tags
        text = re.sub(r'<[^>]+>', '', text)

        # 11. Remove invalid characters (non-ASCII and words containing unusual symbols)
        text = re.sub(r'[^\x00-\x7F]+', '', text)

        # 12. Clean up formatting
        text = re.sub(r'\s+', ' ', text)  # Replace multiple whitespace with a single space
        text = re.sub(r'(?<=[a-zA-Z])(?=\d)', ' ', text)  # Add space before digits

        return text

    return clean_text(title), clean_text(content)

In [34]:
def extract_article_id(link):
    match = re.search(r'articleshow/(\d+)\.cms', link)
    if match:
        return match.group(1)
    return None

In [47]:
print("Extracting articles...")
news = []

for url in tqdm(df["link"]):

    article_id = extract_article_id(url)
    if not article_id:
        news.append(["", ""])

    try:

        headers = {
            "User-Agent": choice(USER_AGENTS)
        }
        proxy_addr = choice(PROXIES)
        proxies = {
            "http": f"http://{proxy_addr}",
            "https": f"http://{proxy_addr}"
        }

        response = get(url, headers=headers, proxies=proxies, timeout=5)
        soup = BeautifulSoup(response.text, 'html.parser')


        try:
            main_div = soup.find('div', id=f'toi-article-container-{article_id}', class_='clearfix rel')
        
            if main_div:
            
                # Extract title from h1 tag with class 'HNMDR'
                title_element = main_div.find('h1', class_='HNMDR')
                title = title_element.get_text(strip=True) if title_element else ""
            
                # Extract content from div with class '_s30J clearfix'
                content_element = main_div.find('div', class_='_s30J clearfix')
                if content_element:
                    content = content_element.get_text(separator=" ", strip=True)
                else:
                    content = ""

            news.append([title, content])
            print(f"[INFO] Article {url} extracted.")
            
        except Exception as E:
            news.append(["", ""])
            print(f"Error fetching {url} inside article : {E}")

    except Exception as e:
        news.append(["", ""])
        print(f"Error fetching {url} : {e}")  



Extracting articles...


  1%|▏         | 1/78 [00:02<03:34,  2.79s/it]

[INFO] Article https://timesofindia.indiatimes.com/first-look/detective-byomkesh-bakshy/articleshow/20874151.cms extracted


  3%|▎         | 2/78 [00:05<03:19,  2.63s/it]

[INFO] Article https://timesofindia.indiatimes.com/first-look/first-look-of-ranbir-kapoors-roy-also-starring-arjun-rampal-and-jacqueline-fernandez/articleshow/21526875.cms extracted


...

[INFO] Article https://timesofindia.indiatimes.com/sports/nfl/news/stefon-diggs-sends-two-word-flirtatious-message-to-cardi-b-amid-backlash-over-viral-yacht-party-video/articleshow/121545177.cms extracted


 99%|█████████▊| 77/78 [02:58<00:02,  2.35s/it]

[INFO] Article https://timesofindia.indiatimes.com/sports/nfl/news/from-dating-rumors-to-i-do-inside-hailee-steinfeld-and-josh-allens-intimate-fairytale-wedding/articleshow/121544868.cms extracted


100%|██████████| 78/78 [03:01<00:00,  2.32s/it]

[INFO] Article https://timesofindia.indiatimes.com/sports/nfl/news/aaron-rodgers-leaves-steelers-hanging-as-potential-signing-remains-shrouded-in-silence-and-speculation/articleshow/121538722.cms extracted


In [63]:
df = pd.read_csv("df.csv")

In [64]:
df["title"], df["content"] = zip(*news)
df

,link,date,publisher,title,content
0,https://timesofindia.indiatimes.com/first-look...,2022-01-01,Times of India,Detective Byomkesh Bakshy!,Credits Director: Dibakar Banerjee Cast: Susha...
1,https://timesofindia.indiatimes.com/first-look...,2022-01-01,Times of India,Roy,Credits Director: Vikramjit Singh Cast: Ranbir...
2,https://timesofindia.indiatimes.com/entertainm...,2022-01-01,Times of India,Edge of Tomorrow,Credits Director: Doug Liman Cast: Tom Cruise ...
3,https://timesofindia.indiatimes.com/sports/cri...,2022-01-01,Times of India,"Bloody! Face covered with towel, Rachin Ravind...",Rachin Ravindra (Screengrab) NEW DELHI: During...
4,https://timesofindia.indiatimes.com/sports/cri...,2022-01-01,Times of India,'Poor lights of Gaddafi Stadium': Pakistan Cri...,Rachin Ravindra (Screengrabs) NEW DELHI: Durin...
...,...,...,...,...,...
73,https://timesofindia.indiatimes.com/sports/nba...,2022-01-03,Times of India,New York Knicks vs Indiana Pacers (05/31): Box...,Tyrese Haliburton's celebration after Indiana ...
74,https://timesofindia.indiatimes.com/sports/nba...,2022-01-03,Times of India,âJust unbelievableâ â Pascal Siakam give...,Pascal Siakam. Image via: AJ Mast/ AP Pascal S...
75,https://timesofindia.indiatimes.com/sports/nfl...,2022-01-03,Times of India,Stefon Diggs sends two-word flirtatious messag...,Stefon Diggs two word flirtatious message to C...
76,https://timesofindia.indiatimes.com/sports/nfl...,2022-01-03,Times of India,From dating rumors to âI doâ: Inside Haile...,Hailee Steinfeld and Josh Allen are finally ma...


In [67]:
df = df[(df["title"] != "") & (df["content"] != "")]

for i in range(len(df)):
    df.loc[i, "title"], df.loc[i, "content"] = process(df.loc[i, "title"], df.loc[i, "content"])

df

,link,date,publisher,title,content
0,https://timesofindia.indiatimes.com/first-look...,2022-01-01,Times of India,Detective Byomkesh Bakshy!,Credits Director: Dibakar Banerjee Cast: Susha...
1,https://timesofindia.indiatimes.com/first-look...,2022-01-01,Times of India,Roy,Credits Director: Vikramjit Singh Cast: Ranbir...
2,https://timesofindia.indiatimes.com/entertainm...,2022-01-01,Times of India,Edge of Tomorrow,Credits Director: Doug Liman Cast: Tom Cruise ...
3,https://timesofindia.indiatimes.com/sports/cri...,2022-01-01,Times of India,"Bloody! Face covered with towel, Rachin Ravind...",Rachin Ravindra (Screengrab) NEW DELHI: During...
4,https://timesofindia.indiatimes.com/sports/cri...,2022-01-01,Times of India,'Poor lights of Gaddafi Stadium': Pakistan Cri...,Rachin Ravindra (Screengrabs) NEW DELHI: Durin...
...,...,...,...,...,...
73,https://timesofindia.indiatimes.com/sports/nba...,2022-01-03,Times of India,New York Knicks vs Indiana Pacers (05/31): Box...,Tyrese Haliburton's celebration after Indiana ...
74,https://timesofindia.indiatimes.com/sports/nba...,2022-01-03,Times of India,Just unbelievable Pascal Siakam gives candid r...,Pascal Siakam. Image via: AJ Mast/ AP Pascal S...
75,https://timesofindia.indiatimes.com/sports/nfl...,2022-01-03,Times of India,Stefon Diggs sends two-word flirtatious messag...,Stefon Diggs two word flirtatious message to C...
76,https://timesofindia.indiatimes.com/sports/nfl...,2022-01-03,Times of India,From dating rumors to I do: Inside Hailee Stei...,Hailee Steinfeld and Josh Allen are finally ma...


In [66]:
df.to_csv("df_processed.csv", index=False)